In [4]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE,ADASYN,SVMSMOTE,BorderlineSMOTE,SMOTENC,KMeansSMOTE
#import tensorflow as tf
import matplotlib.pyplot as plt
#from sklearn.feature_selection import SelectKBest,chi2,SequentialFeatureSelector as sfs
from math import log
from imblearn.combine import SMOTETomek,SMOTEENN
from imblearn.under_sampling import TomekLinks

In [5]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [6]:
### Loading the required files

In [7]:
pickle_in = open("Final_Data_.pickle","rb")
Final_Data = pickle.load(pickle_in)
pickle_in.close()

In [8]:
### Split data into Train, Test and Validation

In [9]:
x = Final_Data.drop('AUDIT_STATUS',axis=1)
y = pd.DataFrame(Final_Data['AUDIT_STATUS'])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0,shuffle=True,stratify=y)

In [12]:
data = pd.DataFrame(x_test['CLAIM_NBR'])
data= data.astype({'CLAIM_NBR':'Int64'})
data.to_csv('Test_Claims.csv',index=True,header=True)

In [13]:
x_train = x_train.drop(['CLAIM_NBR'],axis=1)
x_test = x_test.drop(['CLAIM_NBR'],axis=1)

In [14]:
## Checking the Selection %

In [15]:
print(y_train['AUDIT_STATUS'].value_counts()/len(y_train)*100) #7.14
print(y_test['AUDIT_STATUS'].value_counts()/len(y_test)*100)   #7.14

0    98.796712
1     1.203288
Name: AUDIT_STATUS, dtype: float64
0    98.793899
1     1.206101
Name: AUDIT_STATUS, dtype: float64


In [16]:
####Data PreProcessing (numeric columns of train_data)

##Separating out Numeric columns from train data

In [17]:
train_num = x_train.select_dtypes(include='number')

In [18]:
#train_num = train_num.drop(['PROV_HIT_RATE'],axis=1)

In [19]:
train_num.dtypes

UN              Float64
MOD_CNT           Int64
NET_PAID_AMT    Float64
CHRG            Float64
MBR_RSP         Float64
DEDUC           Float64
MBR_AGE           Int64
DIAG_CNT          Int64
dtype: object

In [20]:
sc = StandardScaler()
train_num_process = pd.DataFrame(sc.fit_transform(train_num))
train_num_process.columns = train_num.columns
train_num_process.index = train_num.index

In [21]:
##PreProcessing numeric data

In [22]:
pickle_out = open('Scaler.pkl','wb')
pickle.dump(sc,pickle_out)
pickle_out.close()

In [23]:
### Combining the processed data to train_data

In [24]:
col_drop = train_num.columns
x_train = x_train.drop(col_drop,axis=1)

In [25]:
## Combining the PreProcessed data to train_data

In [26]:
x_train = pd.concat([x_train,train_num_process],axis=1)

In [27]:
x_train = x_train.astype({'LINE_ITEM_CURRENT_ORIGINAL'                   :'Int64',
'LINE_ITEM_REWORK_ADD_PAY'                     :'Int64',
'LINE_ITEM_OTHER_INSURANCE'                    :'Int64',
'LINE_ITEM_ORIGINAL_BACKED_OUT'               :'Int64',
'LINE_ITEM_REVERSAL_LINE'                      :'Int64',
'LINE_ITEM_HISTORY_LINE'                       :'Int64',
'LINE_ITEM_OTHERS'                             :'Int64',
'CIS_CAT_MISSING'                              :'Int64',
'CIS_CAT_SERVICES_D'                           :'Int64',
'CIS_CAT_E03'                                  :'Int64',
'CIS_CAT_UNIQUE_SER'                           :'Int64',
'CIS_CAT_SKILLED_NU'                           :'Int64',
'CIS_CAT_E01'                                  :'Int64',
'CIS_CAT_PHYSICAL_T'                           :'Int64',
'CIS_CAT_CLINIC'                               :'Int64',
'CIS_CAT_OCCUPATION'                           :'Int64',
'CIS_CAT_CLINIC_I'                             :'Int64',
'CIS_CAT_NON_SURG_P'                           :'Int64',
'CIS_CAT_OTHERS'                               :'Int64',
'OVR_00'                                       :'Int64',
'OVR_27'                                       :'Int64',
'OVR_01'                                       :'Int64',
'OVR_37'                                       :'Int64',
'OVR_13'                                       :'Int64',
'OVR_OTHERS'                                   :'Int64',
'REV_MISSING'                                  :'Int64',
'REV_0300'                                     :'Int64',
'REV_0636'                                     :'Int64',
'REV_0301'                                     :'Int64',
'REV_0023'                                     :'Int64',
'REV_0521'                                     :'Int64',
'REV_0305'                                     :'Int64',
'REV_0250'                                     :'Int64',
'REV_0450'                                     :'Int64',
'REV_0551'                                     :'Int64',
'REV_0510'                                     :'Int64',
'REV_0421'                                     :'Int64',
'REV_0730'                                     :'Int64',
'REV_0260'                                     :'Int64',
'REV_0306'                                     :'Int64',
'REV_0320'                                     :'Int64',
'REV_0272'                                     :'Int64',
'REV_0324'                                     :'Int64',
'REV_0258'                                     :'Int64',
'REV_0983'                                     :'Int64',
'REV_0981'                                     :'Int64',
'REV_0270'                                     :'Int64',
'REV_0420'                                     :'Int64',
'REV_0307'                                     :'Int64',
'REV_0490'                                     :'Int64',
'REV_0370'                                     :'Int64',
'REV_0431'                                     :'Int64',
'REV_0360'                                     :'Int64',
'REV_0710'                                     :'Int64',
'REV_0352'                                     :'Int64',
'REV_0302'                                     :'Int64',
'REV_0424'                                     :'Int64',
'REV_0762'                                     :'Int64',
'REV_0960'                                     :'Int64',
'REV_0637'                                     :'Int64',
'REV_0761'                                     :'Int64',
'REV_0351'                                     :'Int64',
'REV_0460'                                     :'Int64',
'REV_0430'                                     :'Int64',
'REV_OTHERS'                                   :'Int64',
'CPT_1'                                        :'Int64',
'CPT_2'                                        :'Int64',
'CPT_3'                                        :'Int64',
'CPT_4'                                        :'Int64',
'CPT_5'                                        :'Int64',
'CPT_6'                                        :'Int64',
'CPT_7'                                        :'Int64',
'CPT_8'                                        :'Int64',
'CPT_9'                                        :'Int64',
'CPT_10'                                       :'Int64',
'CPT_11'                                       :'Int64',
'CPT_12'                                       :'Int64',
'CPT_13'                                       :'Int64',
'CPT_MISSING'                                  :'Int64',
'CPT_14'                                       :'Int64',
'CPT_15'                                       :'Int64',
'CPT_16'                                       :'Int64',
'CPT_OTHERS'                                   :'Int64',
'SFX_MISSING'                                  :'Int64',
'SFX_1'                                        :'Int64',
'SFX_2'                                        :'Int64',
'SFX_3'                                        :'Int64',
'SFX_4'                                        :'Int64',
'SFX_5'                                        :'Int64',
'SFX_6'                                        :'Int64',
'SFX_7'                                        :'Int64',
'SFX_8'                                        :'Int64',
'SFX_9'                                        :'Int64',
'SFX_10'                                       :'Int64',
'SFX_11'                                       :'Int64',
'SFX_12'                                       :'Int64',
'SFX_13'                                       :'Int64',
'SFX_14'                                       :'Int64',
'SFX_15'                                       :'Int64',
'SFX_16'                                       :'Int64',
'SFX_17'                                       :'Int64',
'SFX_18'                                       :'Int64',
'SFX_OTHERS'                                   :'Int64',
'REJ_EX_CD_MISSING'                            :'Int64',
'REJ_EX_CD_1'                                  :'Int64',
'REJ_EX_CD_2'                                  :'Int64',
'REJ_EX_CD_3'                                  :'Int64',
'REJ_EX_CD_4'                                  :'Int64',
'REJ_EX_CD_5'                                  :'Int64',
'REJ_EX_CD_6'                                  :'Int64',
'REJ_EX_CD_7'                                  :'Int64',
'REJ_EX_CD_8'                                  :'Int64',
'REJ_EX_CD_9'                                  :'Int64',
'REJ_EX_CD_10'                                 :'Int64',
'REJ_EX_CD_11'                                 :'Int64',
'REJ_EX_CD_12'                                 :'Int64',
'REJ_EX_CD_13'                                 :'Int64',
'REJ_EX_CD_14'                                 :'Int64',
'REJ_EX_CD_15'                                 :'Int64',
'REJ_EX_CD_16'                                 :'Int64',
'REJ_EX_CD_17'                                 :'Int64',
'REJ_EX_CD_18'                                 :'Int64',
'REJ_EX_CD_19'                                 :'Int64',
'REJ_EX_CD_20'                                 :'Int64',
'REJ_EX_CD_21'                                 :'Int64',
'REJ_EX_CD_22'                                 :'Int64',
'REJ_EX_CD_23'                                 :'Int64',
'REJ_EX_CD_24'                                 :'Int64',
'REJ_EX_CD_25'                                 :'Int64',
'REJ_EX_CD_26'                                 :'Int64',
'REJ_EX_CD_27'                                 :'Int64',
'REJ_EX_CD_28'                                 :'Int64',
'REJ_EX_CD_29'                                 :'Int64',
'REJ_EX_CD_30'                                 :'Int64',
'REJ_EX_CD_31'                                 :'Int64',
'REJ_EX_CD_32'                                 :'Int64',
'REJ_EX_CD_33'                                 :'Int64',
'REJ_EX_CD_34'                                 :'Int64',
'REJ_EX_CD_35'                                 :'Int64',
'REJ_EX_CD_36'                                 :'Int64',
'REJ_EX_CD_37'                                 :'Int64',
'REJ_EX_CD_38'                                 :'Int64',
'REJ_EX_CD_OTHERS'                             :'Int64',
'POT_22'                                       :'Int64',
'POT_11'                                       :'Int64',
'POT_12'                                       :'Int64',
'POT_23'                                       :'Int64',
'POT_21'                                       :'Int64',
'POT_24'                                       :'Int64',
'POT_81'                                       :'Int64',
'POT_31'                                       :'Int64',
'POT_32'                                       :'Int64',
'POT_99'                                       :'Int64',
'POT_41'                                       :'Int64',
'POT_20'                                       :'Int64',
'POT_19'                                       :'Int64',
'POT_02'                                       :'Int64',
'POT_OTHERS'                                   :'Int64',
'PYEE_R'                                       :'Int64',
'PYEE_5'                                       :'Int64',
'PYEE_2'                                       :'Int64',
'PYEE_E'                                       :'Int64',
'PYEE_4'                                       :'Int64',
'PYEE_H'                                       :'Int64',
'PYEE_S'                                       :'Int64',
'PYEE_I'                                       :'Int64',
'PYEE_MISSING'                                 :'Int64',
'PYEE_OTHERS'                                  :'Int64',
'CAS_CAUSE_GENERAL_ILLNESS'                    :'Int64',
'CAS_CAUSE_MISSING'                            :'Int64',
'CAS_CAUSE_ROUTINE_CARE'                       :'Int64',
'CAS_CAUSE_MENTAL_AND_NERVOUS'                 :'Int64',
'CAS_CAUSE_ACCIDENT'                           :'Int64',
'CAS_CAUSE_ORGAN_TRANSPLANT'                  :'Int64',
'CAS_CAUSE_BIRTH_CONTROL_FAMILY_PLANNING'      :'Int64',
'CAS_CAUSE_SERV_ARENOT_COVERED_BASED_ON_DIAG_BILLED'    :'Int64',
'CAS_CAUSE_ROUTINE_VISION'    :'Int64',
'CAS_CAUSE_DRUG_ABUSE'    :'Int64',
'CAS_CAUSE_OTHERS'                             :'Int64',
'SUSP_K'                                       :'Int64',
'SUSP_MISSING'                                 :'Int64',
'SUSP_Q'                                       :'Int64',
'SUSP_P'                                       :'Int64',
'SUSP_D'                                       :'Int64',
'SUSP_I'                                       :'Int64',
'SUSP_Y'                                       :'Int64',
'SUSP_OTHERS'                                  :'Int64'})

In [28]:
## performing one-hot-encoding on categorical features

In [29]:
ctgrl_col = pd.DataFrame(x_train[['PRCSD_AS', 'LOB', 'NTWK', 'PRV_SPC', 'TOB', 'PAR_CD', 'VCH', 'MBR_ST', 'PRIM_DX_MAPPING', 'SUSDUP', 'MSL_INDICATOR', 'REFERRAL', 'CLI_BFT','PROCESSED_GROUP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_train = x_train.drop(ctgrl_col.columns,axis=1)
x_train = pd.concat([x_train,ctgrl_cols],axis=1)

In [30]:
### Data balancing using SMOTE

In [31]:
y_train.value_counts(normalize=True)*100

AUDIT_STATUS
0               98.796712
1                1.203288
dtype: float64

In [32]:
sm = SMOTE(sampling_strategy=0.43,random_state=0,n_jobs=-1)

In [33]:
x_sm,y_sm = sm.fit_resample(x_train.values,y_train.values.ravel())

In [34]:
x_sm = pd.DataFrame(x_sm)
y_sm = pd.DataFrame(y_sm)

In [35]:
x_sm.columns = x_train.columns
y_sm.columns = y_train.columns

In [36]:
y_sm.value_counts(normalize=True)*100

AUDIT_STATUS
0               69.930524
1               30.069476
dtype: float64

In [37]:
####################################################
####################################################
####################################################

### Model Building

In [32]:
x_sm = pd.DataFrame(x_sm[['SERV_STATE_CD_IL',
'PYMT_CAT_CD_E',
'PYMT_CAT_CD_C',
'SERV_STATE_CD_UT',
'REV_0023',
'HIPPS_PA_MISSING',
'SERV_STATE_CD_MO',
'SRC_PAR_CD_L',
'SRC_PAR_CD_5',
'HIPPS_PA_19',
'SERV_STATE_CD_WI',
'SERV_STATE_CD_KY',
'SERV_STATE_CD_CA',
'SERV_STATE_CD_PA',
'SRC_PAR_CD_X',
'SERV_STATE_CD_AL',
'SERV_STATE_CD_OH',
'SRC_NTWK_ID_7030H3I',
'HCPCS_UN_2',
'SERV_STATE_CD_VA',
'SERV_STATE_CD_NC',
'SRC_NTWK_ID_5367H3I',
'HCPCS_PA_MISSING',
'HCPCS_CAMT_2',
'SRC_NTWK_ID_5366H32',
'HIPPS_UN_5',
'SERV_STATE_CD_MN',
'SRC_NTWK_ID_7037H32',
'SERV_STATE_CD_OTHERS',
'SERV_STATE_CD_KS',
'HIPPS_UN_10',
'SERV_STATE_CD_LA',
'SERV_STATE_CD_NY',
'SERV_STATE_CD_WV',
'SERV_STATE_CD_MS',
'CAUSE_CD_001',
'SERV_STATE_CD_TN',
'HCPCS_UN_MISSING',
'SERV_STATE_CD_WA',
'SERV_STATE_CD_OK',
'SERV_STATE_CD_SC',
'HCPCS_CAMT_1',
'BILL_TYPE_CD_327',
'SRC_NTWK_ID_5360H3G',
'SERV_STATE_CD_MI',
'REV_0551',
'CLM_BENEFIT_PAR_LCLM_CAT_CD_N',
'HCPCS_PA_2',
'ADMIT_SRC_CD_1',
'SERV_STATE_CD_TX',
'SERV_STATE_CD_GA',
'REV_0272',
'PRIMARY_DIAG_MAPPING_2',
'REV_0623',
'HCPCS_CAMT_MISSING',
'BILL_TYPE_CD_329',
'HCPCS_PA_1',
'REV_0550',
'PRIMARY_DIAG_MAPPING_9',
'HIPPS_PA_12',
'SRC_NTWK_ID_0000H32',
'REV_0270',
'LINE_CAT_CD_3100',
'PRIMARY_DIAG_MAPPING_8',
'HIPPS_UN_1',
'SERV_STATE_CD_IN',
'SERV_TYPE_CD1_CNT',
'ADMIT_SRC_CD_5',
'HIPPS_PA_OTHERS',
'HIPPS_UN_4',
'LINE_CAT_CD_3030',
'SRC_NTWK_ID_5241H3I',
'HIPPS_CAMT_MISSING',
'REV_0421',
'HIPPS_PA_10',
'HIPPS_PA_3',
'SRC_PAR_CD_OTHERS',
'PRIMARY_DIAG_MAPPING_4',
'HCPCS_UN_1',
'HIPPS_CAMT_1',
'ADMIT_SRC_CD_2',
'ADMIT_SRC_CD_4',
'HIPPS_PA_17',
'HIPPS_PA_2',
'CAUSE_CD_010',
'HCPCS_UN_3',
'DIAG_CNT',
'PRIMARY_DIAG_MAPPING_5',
'HIPPS_UN_MISSING',
'PRIMARY_DIAG_MAPPING_7',
'REV_0431',
'HIPPS_PA_5',
'HIPPS_PA_1',
'CAUSE_CD_002',
'CAUSE_CD_MISSING',
'CLM_BENEFIT_PAR_LCLM_CAT_CD_P',
'HIPPS_UN_12',
'CLM_BENEFIT_PAR_CAT_CD_P',
'DISCHG_STAT_CD_01',
'REV_0420',
'HIPPS_CAMT_OTHERS',
'HIPPS_UN_2',
'ADMIT_SRC_CD_6',
'SERV_STATE_CD_MT',
'PRIMARY_DIAG_MAPPING_11',
'DISCHG_STAT_CD_02',
'PRIMARY_DIAG_MAPPING_3',
'REV_0430',
'SERV_STATE_CD_FL',
'PRIMARY_DIAG_MAPPING_10',
'HIPPS_PA_4',
'PRIMARY_DIAG_MAPPING_13',
'SRC_NTWK_ID_0000H39',
'DISCHG_STAT_CD_30',
'CLM_BENEFIT_PAR_CAT_CD_N',
'REV_0561',
'PRIMARY_DIAG_MAPPING_1',
'REV_0571',
'HIPPS_PA_18',
'DISCHG_STAT_CD_OTHERS',
'HIPPS_PA_14',
'HIPPS_PA_9',
'SRC_NTWK_ID_OTHERS',
'PRIMARY_DIAG_MAPPING_12',
'REV_OTHERS',
'HIPPS_PA_16',
'HIPPS_CAMT_3',
'HIPPS_UN_13',
'HIPPS_PA_6',
'HIPPS_UN_OTHERS',
'SRC_NTWK_ID_5430H3G',
'HIPPS_PA_7',
'SERV_STATE_CD_NH',
'HIPPS_PA_8',
'REV_0441',
'HIPPS_PA_13',
'HIPPS_CAMT_2',
'HCPCS_PA_3',
'DISCHG_STAT_CD_50',
'HIPPS_UN_16'
]])

In [38]:
#####################################

####Data PreProcessing (numeric columns of test_data)

##Separating out Numeric columns from test data

In [39]:
test_num = x_test.select_dtypes(include='number')

In [40]:
#test_num = test_num.drop(['PROV_HIT_RATE'],axis=1)

In [41]:
##PreProcessing numeric data

In [42]:
test_num_process = pd.DataFrame(sc.transform(test_num))
test_num_process.columns = test_num.columns
test_num_process.index = test_num.index

In [43]:
col_drop = test_num.columns
x_test = x_test.drop(col_drop,axis=1)

In [44]:
x_test = pd.concat([x_test,test_num_process],axis=1)

In [45]:
x_test = x_test.astype({'LINE_ITEM_CURRENT_ORIGINAL'                   :'Int64',
'LINE_ITEM_REWORK_ADD_PAY'                     :'Int64',
'LINE_ITEM_OTHER_INSURANCE'                    :'Int64',
'LINE_ITEM_ORIGINAL_BACKED_OUT'               :'Int64',
'LINE_ITEM_REVERSAL_LINE'                      :'Int64',
'LINE_ITEM_HISTORY_LINE'                       :'Int64',
'LINE_ITEM_OTHERS'                             :'Int64',
'CIS_CAT_MISSING'                              :'Int64',
'CIS_CAT_SERVICES_D'                           :'Int64',
'CIS_CAT_E03'                                  :'Int64',
'CIS_CAT_UNIQUE_SER'                           :'Int64',
'CIS_CAT_SKILLED_NU'                           :'Int64',
'CIS_CAT_E01'                                  :'Int64',
'CIS_CAT_PHYSICAL_T'                           :'Int64',
'CIS_CAT_CLINIC'                               :'Int64',
'CIS_CAT_OCCUPATION'                           :'Int64',
'CIS_CAT_CLINIC_I'                             :'Int64',
'CIS_CAT_NON_SURG_P'                           :'Int64',
'CIS_CAT_OTHERS'                               :'Int64',
'OVR_00'                                       :'Int64',
'OVR_27'                                       :'Int64',
'OVR_01'                                       :'Int64',
'OVR_37'                                       :'Int64',
'OVR_13'                                       :'Int64',
'OVR_OTHERS'                                   :'Int64',
'REV_MISSING'                                  :'Int64',
'REV_0300'                                     :'Int64',
'REV_0636'                                     :'Int64',
'REV_0301'                                     :'Int64',
'REV_0023'                                     :'Int64',
'REV_0521'                                     :'Int64',
'REV_0305'                                     :'Int64',
'REV_0250'                                     :'Int64',
'REV_0450'                                     :'Int64',
'REV_0551'                                     :'Int64',
'REV_0510'                                     :'Int64',
'REV_0421'                                     :'Int64',
'REV_0730'                                     :'Int64',
'REV_0260'                                     :'Int64',
'REV_0306'                                     :'Int64',
'REV_0320'                                     :'Int64',
'REV_0272'                                     :'Int64',
'REV_0324'                                     :'Int64',
'REV_0258'                                     :'Int64',
'REV_0983'                                     :'Int64',
'REV_0981'                                     :'Int64',
'REV_0270'                                     :'Int64',
'REV_0420'                                     :'Int64',
'REV_0307'                                     :'Int64',
'REV_0490'                                     :'Int64',
'REV_0370'                                     :'Int64',
'REV_0431'                                     :'Int64',
'REV_0360'                                     :'Int64',
'REV_0710'                                     :'Int64',
'REV_0352'                                     :'Int64',
'REV_0302'                                     :'Int64',
'REV_0424'                                     :'Int64',
'REV_0762'                                     :'Int64',
'REV_0960'                                     :'Int64',
'REV_0637'                                     :'Int64',
'REV_0761'                                     :'Int64',
'REV_0351'                                     :'Int64',
'REV_0460'                                     :'Int64',
'REV_0430'                                     :'Int64',
'REV_OTHERS'                                   :'Int64',
'CPT_1'                                        :'Int64',
'CPT_2'                                        :'Int64',
'CPT_3'                                        :'Int64',
'CPT_4'                                        :'Int64',
'CPT_5'                                        :'Int64',
'CPT_6'                                        :'Int64',
'CPT_7'                                        :'Int64',
'CPT_8'                                        :'Int64',
'CPT_9'                                        :'Int64',
'CPT_10'                                       :'Int64',
'CPT_11'                                       :'Int64',
'CPT_12'                                       :'Int64',
'CPT_13'                                       :'Int64',
'CPT_MISSING'                                  :'Int64',
'CPT_14'                                       :'Int64',
'CPT_15'                                       :'Int64',
'CPT_16'                                       :'Int64',
'CPT_OTHERS'                                   :'Int64',
'SFX_MISSING'                                  :'Int64',
'SFX_1'                                        :'Int64',
'SFX_2'                                        :'Int64',
'SFX_3'                                        :'Int64',
'SFX_4'                                        :'Int64',
'SFX_5'                                        :'Int64',
'SFX_6'                                        :'Int64',
'SFX_7'                                        :'Int64',
'SFX_8'                                        :'Int64',
'SFX_9'                                        :'Int64',
'SFX_10'                                       :'Int64',
'SFX_11'                                       :'Int64',
'SFX_12'                                       :'Int64',
'SFX_13'                                       :'Int64',
'SFX_14'                                       :'Int64',
'SFX_15'                                       :'Int64',
'SFX_16'                                       :'Int64',
'SFX_17'                                       :'Int64',
'SFX_18'                                       :'Int64',
'SFX_OTHERS'                                   :'Int64',
'REJ_EX_CD_MISSING'                            :'Int64',
'REJ_EX_CD_1'                                  :'Int64',
'REJ_EX_CD_2'                                  :'Int64',
'REJ_EX_CD_3'                                  :'Int64',
'REJ_EX_CD_4'                                  :'Int64',
'REJ_EX_CD_5'                                  :'Int64',
'REJ_EX_CD_6'                                  :'Int64',
'REJ_EX_CD_7'                                  :'Int64',
'REJ_EX_CD_8'                                  :'Int64',
'REJ_EX_CD_9'                                  :'Int64',
'REJ_EX_CD_10'                                 :'Int64',
'REJ_EX_CD_11'                                 :'Int64',
'REJ_EX_CD_12'                                 :'Int64',
'REJ_EX_CD_13'                                 :'Int64',
'REJ_EX_CD_14'                                 :'Int64',
'REJ_EX_CD_15'                                 :'Int64',
'REJ_EX_CD_16'                                 :'Int64',
'REJ_EX_CD_17'                                 :'Int64',
'REJ_EX_CD_18'                                 :'Int64',
'REJ_EX_CD_19'                                 :'Int64',
'REJ_EX_CD_20'                                 :'Int64',
'REJ_EX_CD_21'                                 :'Int64',
'REJ_EX_CD_22'                                 :'Int64',
'REJ_EX_CD_23'                                 :'Int64',
'REJ_EX_CD_24'                                 :'Int64',
'REJ_EX_CD_25'                                 :'Int64',
'REJ_EX_CD_26'                                 :'Int64',
'REJ_EX_CD_27'                                 :'Int64',
'REJ_EX_CD_28'                                 :'Int64',
'REJ_EX_CD_29'                                 :'Int64',
'REJ_EX_CD_30'                                 :'Int64',
'REJ_EX_CD_31'                                 :'Int64',
'REJ_EX_CD_32'                                 :'Int64',
'REJ_EX_CD_33'                                 :'Int64',
'REJ_EX_CD_34'                                 :'Int64',
'REJ_EX_CD_35'                                 :'Int64',
'REJ_EX_CD_36'                                 :'Int64',
'REJ_EX_CD_37'                                 :'Int64',
'REJ_EX_CD_38'                                 :'Int64',
'REJ_EX_CD_OTHERS'                             :'Int64',
'POT_22'                                       :'Int64',
'POT_11'                                       :'Int64',
'POT_12'                                       :'Int64',
'POT_23'                                       :'Int64',
'POT_21'                                       :'Int64',
'POT_24'                                       :'Int64',
'POT_81'                                       :'Int64',
'POT_31'                                       :'Int64',
'POT_32'                                       :'Int64',
'POT_99'                                       :'Int64',
'POT_41'                                       :'Int64',
'POT_20'                                       :'Int64',
'POT_19'                                       :'Int64',
'POT_02'                                       :'Int64',
'POT_OTHERS'                                   :'Int64',
'PYEE_R'                                       :'Int64',
'PYEE_5'                                       :'Int64',
'PYEE_2'                                       :'Int64',
'PYEE_E'                                       :'Int64',
'PYEE_4'                                       :'Int64',
'PYEE_H'                                       :'Int64',
'PYEE_S'                                       :'Int64',
'PYEE_I'                                       :'Int64',
'PYEE_MISSING'                                 :'Int64',
'PYEE_OTHERS'                                  :'Int64',
'CAS_CAUSE_GENERAL_ILLNESS'                    :'Int64',
'CAS_CAUSE_MISSING'                            :'Int64',
'CAS_CAUSE_ROUTINE_CARE'                       :'Int64',
'CAS_CAUSE_MENTAL_AND_NERVOUS'                 :'Int64',
'CAS_CAUSE_ACCIDENT'                           :'Int64',
'CAS_CAUSE_ORGAN_TRANSPLANT'                  :'Int64',
'CAS_CAUSE_BIRTH_CONTROL_FAMILY_PLANNING'      :'Int64',
'CAS_CAUSE_SERV_ARENOT_COVERED_BASED_ON_DIAG_BILLED'    :'Int64',
'CAS_CAUSE_ROUTINE_VISION'    :'Int64',
'CAS_CAUSE_DRUG_ABUSE'    :'Int64',
'CAS_CAUSE_OTHERS'                             :'Int64',
'SUSP_K'                                       :'Int64',
'SUSP_MISSING'                                 :'Int64',
'SUSP_Q'                                       :'Int64',
'SUSP_P'                                       :'Int64',
'SUSP_D'                                       :'Int64',
'SUSP_I'                                       :'Int64',
'SUSP_Y'                                       :'Int64',
'SUSP_OTHERS'                                  :'Int64'})

In [46]:
###########
### Combining the processed data to test_data

In [47]:
ctgrl_col = pd.DataFrame(x_test[['PRCSD_AS', 'LOB', 'NTWK', 'PRV_SPC', 'TOB', 'PAR_CD', 'VCH', 'MBR_ST', 'PRIM_DX_MAPPING', 'SUSDUP', 'MSL_INDICATOR', 'REFERRAL', 'CLI_BFT','PROCESSED_GROUP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_test = x_test.drop(ctgrl_col.columns,axis=1)
x_test = pd.concat([x_test,ctgrl_cols],axis=1)

In [45]:
x_test = pd.DataFrame(x_test[['SERV_STATE_CD_IL',
'PYMT_CAT_CD_E',
'PYMT_CAT_CD_C',
'SERV_STATE_CD_UT',
'REV_0023',
'HIPPS_PA_MISSING',
'SERV_STATE_CD_MO',
'SRC_PAR_CD_L',
'SRC_PAR_CD_5',
'HIPPS_PA_19',
'SERV_STATE_CD_WI',
'SERV_STATE_CD_KY',
'SERV_STATE_CD_CA',
'SERV_STATE_CD_PA',
'SRC_PAR_CD_X',
'SERV_STATE_CD_AL',
'SERV_STATE_CD_OH',
'SRC_NTWK_ID_7030H3I',
'HCPCS_UN_2',
'SERV_STATE_CD_VA',
'SERV_STATE_CD_NC',
'SRC_NTWK_ID_5367H3I',
'HCPCS_PA_MISSING',
'HCPCS_CAMT_2',
'SRC_NTWK_ID_5366H32',
'HIPPS_UN_5',
'SERV_STATE_CD_MN',
'SRC_NTWK_ID_7037H32',
'SERV_STATE_CD_OTHERS',
'SERV_STATE_CD_KS',
'HIPPS_UN_10',
'SERV_STATE_CD_LA',
'SERV_STATE_CD_NY',
'SERV_STATE_CD_WV',
'SERV_STATE_CD_MS',
'CAUSE_CD_001',
'SERV_STATE_CD_TN',
'HCPCS_UN_MISSING',
'SERV_STATE_CD_WA',
'SERV_STATE_CD_OK',
'SERV_STATE_CD_SC',
'HCPCS_CAMT_1',
'BILL_TYPE_CD_327',
'SRC_NTWK_ID_5360H3G',
'SERV_STATE_CD_MI',
'REV_0551',
'CLM_BENEFIT_PAR_LCLM_CAT_CD_N',
'HCPCS_PA_2',
'ADMIT_SRC_CD_1',
'SERV_STATE_CD_TX',
'SERV_STATE_CD_GA',
'REV_0272',
'PRIMARY_DIAG_MAPPING_2',
'REV_0623',
'HCPCS_CAMT_MISSING',
'BILL_TYPE_CD_329',
'HCPCS_PA_1',
'REV_0550',
'PRIMARY_DIAG_MAPPING_9',
'HIPPS_PA_12',
'SRC_NTWK_ID_0000H32',
'REV_0270',
'LINE_CAT_CD_3100',
'PRIMARY_DIAG_MAPPING_8',
'HIPPS_UN_1',
'SERV_STATE_CD_IN',
'SERV_TYPE_CD1_CNT',
'ADMIT_SRC_CD_5',
'HIPPS_PA_OTHERS',
'HIPPS_UN_4',
'LINE_CAT_CD_3030',
'SRC_NTWK_ID_5241H3I',
'HIPPS_CAMT_MISSING',
'REV_0421',
'HIPPS_PA_10',
'HIPPS_PA_3',
'SRC_PAR_CD_OTHERS',
'PRIMARY_DIAG_MAPPING_4',
'HCPCS_UN_1',
'HIPPS_CAMT_1',
'ADMIT_SRC_CD_2',
'ADMIT_SRC_CD_4',
'HIPPS_PA_17',
'HIPPS_PA_2',
'CAUSE_CD_010',
'HCPCS_UN_3',
'DIAG_CNT',
'PRIMARY_DIAG_MAPPING_5',
'HIPPS_UN_MISSING',
'PRIMARY_DIAG_MAPPING_7',
'REV_0431',
'HIPPS_PA_5',
'HIPPS_PA_1',
'CAUSE_CD_002',
'CAUSE_CD_MISSING',
'CLM_BENEFIT_PAR_LCLM_CAT_CD_P',
'HIPPS_UN_12',
'CLM_BENEFIT_PAR_CAT_CD_P',
'DISCHG_STAT_CD_01',
'REV_0420',
'HIPPS_CAMT_OTHERS',
'HIPPS_UN_2',
'ADMIT_SRC_CD_6',
'SERV_STATE_CD_MT',
'PRIMARY_DIAG_MAPPING_11',
'DISCHG_STAT_CD_02',
'PRIMARY_DIAG_MAPPING_3',
'REV_0430',
'SERV_STATE_CD_FL',
'PRIMARY_DIAG_MAPPING_10',
'HIPPS_PA_4',
'PRIMARY_DIAG_MAPPING_13',
'SRC_NTWK_ID_0000H39',
'DISCHG_STAT_CD_30',
'CLM_BENEFIT_PAR_CAT_CD_N',
'REV_0561',
'PRIMARY_DIAG_MAPPING_1',
'REV_0571',
'HIPPS_PA_18',
'DISCHG_STAT_CD_OTHERS',
'HIPPS_PA_14',
'HIPPS_PA_9',
'SRC_NTWK_ID_OTHERS',
'PRIMARY_DIAG_MAPPING_12',
'REV_OTHERS',
'HIPPS_PA_16',
'HIPPS_CAMT_3',
'HIPPS_UN_13',
'HIPPS_PA_6',
'HIPPS_UN_OTHERS',
'SRC_NTWK_ID_5430H3G',
'HIPPS_PA_7',
'SERV_STATE_CD_NH',
'HIPPS_PA_8',
'REV_0441',
'HIPPS_PA_13',
'HIPPS_CAMT_2',
'HCPCS_PA_3',
'DISCHG_STAT_CD_50',
'HIPPS_UN_16'
]])

In [46]:
x_train = pd.DataFrame(x_train[['SERV_STATE_CD_IL',
'PYMT_CAT_CD_E',
'PYMT_CAT_CD_C',
'SERV_STATE_CD_UT',
'REV_0023',
'HIPPS_PA_MISSING',
'SERV_STATE_CD_MO',
'SRC_PAR_CD_L',
'SRC_PAR_CD_5',
'HIPPS_PA_19',
'SERV_STATE_CD_WI',
'SERV_STATE_CD_KY',
'SERV_STATE_CD_CA',
'SERV_STATE_CD_PA',
'SRC_PAR_CD_X',
'SERV_STATE_CD_AL',
'SERV_STATE_CD_OH',
'SRC_NTWK_ID_7030H3I',
'HCPCS_UN_2',
'SERV_STATE_CD_VA',
'SERV_STATE_CD_NC',
'SRC_NTWK_ID_5367H3I',
'HCPCS_PA_MISSING',
'HCPCS_CAMT_2',
'SRC_NTWK_ID_5366H32',
'HIPPS_UN_5',
'SERV_STATE_CD_MN',
'SRC_NTWK_ID_7037H32',
'SERV_STATE_CD_OTHERS',
'SERV_STATE_CD_KS',
'HIPPS_UN_10',
'SERV_STATE_CD_LA',
'SERV_STATE_CD_NY',
'SERV_STATE_CD_WV',
'SERV_STATE_CD_MS',
'CAUSE_CD_001',
'SERV_STATE_CD_TN',
'HCPCS_UN_MISSING',
'SERV_STATE_CD_WA',
'SERV_STATE_CD_OK',
'SERV_STATE_CD_SC',
'HCPCS_CAMT_1',
'BILL_TYPE_CD_327',
'SRC_NTWK_ID_5360H3G',
'SERV_STATE_CD_MI',
'REV_0551',
'CLM_BENEFIT_PAR_LCLM_CAT_CD_N',
'HCPCS_PA_2',
'ADMIT_SRC_CD_1',
'SERV_STATE_CD_TX',
'SERV_STATE_CD_GA',
'REV_0272',
'PRIMARY_DIAG_MAPPING_2',
'REV_0623',
'HCPCS_CAMT_MISSING',
'BILL_TYPE_CD_329',
'HCPCS_PA_1',
'REV_0550',
'PRIMARY_DIAG_MAPPING_9',
'HIPPS_PA_12',
'SRC_NTWK_ID_0000H32',
'REV_0270',
'LINE_CAT_CD_3100',
'PRIMARY_DIAG_MAPPING_8',
'HIPPS_UN_1',
'SERV_STATE_CD_IN',
'SERV_TYPE_CD1_CNT',
'ADMIT_SRC_CD_5',
'HIPPS_PA_OTHERS',
'HIPPS_UN_4',
'LINE_CAT_CD_3030',
'SRC_NTWK_ID_5241H3I',
'HIPPS_CAMT_MISSING',
'REV_0421',
'HIPPS_PA_10',
'HIPPS_PA_3',
'SRC_PAR_CD_OTHERS',
'PRIMARY_DIAG_MAPPING_4',
'HCPCS_UN_1',
'HIPPS_CAMT_1',
'ADMIT_SRC_CD_2',
'ADMIT_SRC_CD_4',
'HIPPS_PA_17',
'HIPPS_PA_2',
'CAUSE_CD_010',
'HCPCS_UN_3',
'DIAG_CNT',
'PRIMARY_DIAG_MAPPING_5',
'HIPPS_UN_MISSING',
'PRIMARY_DIAG_MAPPING_7',
'REV_0431',
'HIPPS_PA_5',
'HIPPS_PA_1',
'CAUSE_CD_002',
'CAUSE_CD_MISSING',
'CLM_BENEFIT_PAR_LCLM_CAT_CD_P',
'HIPPS_UN_12',
'CLM_BENEFIT_PAR_CAT_CD_P',
'DISCHG_STAT_CD_01',
'REV_0420',
'HIPPS_CAMT_OTHERS',
'HIPPS_UN_2',
'ADMIT_SRC_CD_6',
'SERV_STATE_CD_MT',
'PRIMARY_DIAG_MAPPING_11',
'DISCHG_STAT_CD_02',
'PRIMARY_DIAG_MAPPING_3',
'REV_0430',
'SERV_STATE_CD_FL',
'PRIMARY_DIAG_MAPPING_10',
'HIPPS_PA_4',
'PRIMARY_DIAG_MAPPING_13',
'SRC_NTWK_ID_0000H39',
'DISCHG_STAT_CD_30',
'CLM_BENEFIT_PAR_CAT_CD_N',
'REV_0561',
'PRIMARY_DIAG_MAPPING_1',
'REV_0571',
'HIPPS_PA_18',
'DISCHG_STAT_CD_OTHERS',
'HIPPS_PA_14',
'HIPPS_PA_9',
'SRC_NTWK_ID_OTHERS',
'PRIMARY_DIAG_MAPPING_12',
'REV_OTHERS',
'HIPPS_PA_16',
'HIPPS_CAMT_3',
'HIPPS_UN_13',
'HIPPS_PA_6',
'HIPPS_UN_OTHERS',
'SRC_NTWK_ID_5430H3G',
'HIPPS_PA_7',
'SERV_STATE_CD_NH',
'HIPPS_PA_8',
'REV_0441',
'HIPPS_PA_13',
'HIPPS_CAMT_2',
'HCPCS_PA_3',
'DISCHG_STAT_CD_50',
'HIPPS_UN_16'
]])

In [48]:
## exporting data

In [49]:
x_sm.to_csv('x_sm.csv',index=False,header=True)
y_sm.to_csv('y_sm.csv',index=False,header=True)
x_train.to_csv('x_train.csv',index=False,header=True)
y_train.to_csv('y_train.csv',index=False,header=True)
x_test.to_csv('x_test.csv',index=False,header=True)
y_train.to_csv('y_test.csv',index=False,header=True)

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE,ADASYN,SVMSMOTE,BorderlineSMOTE,SMOTENC,KMeansSMOTE
import tensorflow as tf
import matplotlib.pyplot as plt
#from sklearn.feature_selection import SelectKBest,chi2,SequentialFeatureSelector as sfs
from math import log
from imblearn.combine import SMOTETomek,SMOTEENN
from imblearn.under_sampling import TomekLinks

C:\Users\KPA2877\.conda\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\KPA2877\.conda\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\KPA2877\.conda\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\KPA2877\.conda\envs\dl\lib\site-packages\tensorflow\python\framework\dtypes.

In [2]:
## importing data

In [3]:
x_sm = pd.read_csv('x_sm.csv') 
y_sm = pd.read_csv('y_sm.csv') 
x_train = pd.read_csv('x_train.csv') 
y_train = pd.read_csv('y_train.csv') 
x_test = pd.read_csv('x_test.csv') 
y_test = pd.read_csv('y_test.csv') 

In [4]:
x_test.shape

(8457, 416)

In [5]:
y_test.shape

(33824, 1)

In [6]:
#Initialising ANN
ann = tf.keras.models.Sequential()

In [7]:
 #Adding First Hidden Layer
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
 #Adding Second Hidden Layer
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

In [9]:
 #Adding Output Layer
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

In [10]:
#Compiling ANN
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [11]:
#Fitting ANN
ann.fit(np.array(x_sm).astype(float),np.array(y_sm).astype(float),batch_size=32,epochs = 100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
47786/47786 [==============================] - 3s 59us/sample - loss: 0.2962 - acc: 0.8695
Epoch 2/100
47786/47786 [==============================] - 3s 56us/sample - loss: 0.2017 - acc: 0.9166
Epoch 3/100
47786/47786 [==============================] - 3s 55us/sample - loss: 0.1724 - acc: 0.9297
Epoch 4/100
47786/47786 [==============================] - 3s 58us/sample - loss: 0.1508 - acc: 0.9396
Epoch 5/100
47786/47786 [==============================] - 3s 57us/sample - loss: 0.1317 - acc: 0.9480
Epoch 6/100
47786/47786 [==============================] - 3s 57us/sample - loss: 0.1144 - acc: 0.9577
Epoch 7/100
47786/47786 [==============================] - 3s 56us/sample - loss: 0.1014 - acc: 0.9650
Epoch 8/100
47786/47786 [==============================] - 3s 55us/sample - loss: 0.0908 - acc: 0.9698
Epoch 9/100
47786/47786 [==============================] - 3s 58us/sample - loss: 

47786/47786 [==============================] - 3s 57us/sample - loss: 0.0130 - acc: 0.99591s - loss: 0.0
Epoch 76/100
47786/47786 [==============================] - 3s 59us/sample - loss: 0.0142 - acc: 0.9951
Epoch 77/100
47786/47786 [==============================] - 3s 56us/sample - loss: 0.0132 - acc: 0.9956
Epoch 78/100
47786/47786 [==============================] - 3s 56us/sample - loss: 0.0137 - acc: 0.9954
Epoch 79/100
47786/47786 [==============================] - 3s 57us/sample - loss: 0.0121 - acc: 0.9961
Epoch 80/100
47786/47786 [==============================] - 3s 60us/sample - loss: 0.0132 - acc: 0.9956
Epoch 81/100
47786/47786 [==============================] - 3s 57us/sample - loss: 0.0114 - acc: 0.9963
Epoch 82/100
47786/47786 [==============================] - 3s 63us/sample - loss: 0.0132 - acc: 0.9956
Epoch 83/100
47786/47786 [==============================] - 3s 59us/sample - loss: 0.0117 - acc: 0.9963
Epoch 84/100
47786/47786 [==============================] - 3s 

In [35]:
## Combining the Preprocessed data to test_data

In [36]:
##################
### Predicting on the test data

In [12]:
y_pred = ann.predict(x_sm.values)
y_pred = pd.DataFrame(np.argmax(y_pred,axis=1))
print(accuracy_score(y_sm,y_pred))
print(confusion_matrix(y_sm,y_pred))

print(classification_report(y_sm,y_pred))

0.699305235843134
[[33417     0]
 [14369     0]]
              precision    recall  f1-score   support

           0       0.70      1.00      0.82     33417
           1       0.00      0.00      0.00     14369

    accuracy                           0.70     47786
   macro avg       0.35      0.50      0.41     47786
weighted avg       0.49      0.70      0.58     47786



C:\Users\KPA2877\.conda\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
y_pred = ann.predict(x_train.values)
y_pred = pd.DataFrame(np.argmax(y_pred,axis=1))
print(accuracy_score(y_train,y_pred))
print(confusion_matrix(y_train,y_pred))

print(classification_report(y_train,y_pred))

0.987967123935667
[[33417     0]
 [  407     0]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     33417
           1       0.00      0.00      0.00       407

    accuracy                           0.99     33824
   macro avg       0.49      0.50      0.50     33824
weighted avg       0.98      0.99      0.98     33824



In [14]:
y_pred = ann.predict(x_test.values)
y_pred = pd.DataFrame(np.argmax(y_pred,axis=1))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [33824, 8457]

In [87]:
pickle_out = open("Model_XGFINAL.pkl","wb")

pickle.dump(XG_v1,pickle_out)

pickle_out.close()

In [88]:
x_sm.shape

(335914, 140)